In [7]:
import sys
import numpy as np
import pandas as pd
import keras


In [8]:
n_user_train = 462
n_user_test = 74
n_sen_train = 1716
n_sen_test = 342

path_data = 'data/'
mfcc_or_fbank = 'mfcc'
if mfcc_or_fbank == 'mfcc' :
    dim = 39
else :
    dim = 69

batch_size = 256

# RNN seting
n_RNN_seq = 3

In [5]:
#
# loading data
#
lab_train_num_reindex_axis = pd.read_csv('{}data_pp/lab_train_num_reindex_axis.csv'.format(path_data))
df_BE_train = pd.read_csv('{}data_pp/beginEnd_train.csv'.format(path_data))
df_BE_test = pd.read_csv('{}data_pp/beginEnd_test.csv'.format(path_data))


print (lab_train_num_reindex_axis.head(3))
print (df_BE_train.head(3))


                0   1
0  faem0_si1392_1  37
1  faem0_si1392_2  37
2  faem0_si1392_3  37
  speakerId sentenceId  index_begin  index_end
0     faem0     si1392            0        473
1     faem0     si2022          474        696
2     faem0      si762          697        970


In [6]:
class RNN_data_generator(object):
    def __init__(self, X, y, BE, n_sen, batch_size=batch_size, n_seq=n_RNN_seq, dim=dim, shuffle = False):
        self.X = X
        self.y = y
        self.BE = BE
        self.n_sen = n_sen
        self.len_X = X.shape[0] - (n_sen * n_seq) + 1
        self.batch_size = batch_size
        self.n_seq = n_seq
        self.dim = dim
        self.shuffle = shuffle

    def generate(self, labels, list_IDs):
        # Generates batches of samples
        # Infinite loop
        while 1:
            # Generate order of exploration of dataset
            indexes = np.arange(self.len_X)
            if self.shuffle == True:
                np.random.shuffle(indexes)
                

            # Generate batches
            imax = int(self.len_X/self.batch_size)
            k_skip = 0
            i_input = 0
            for i_output_batch in range(imax):
                output_X = np.empty((self.batch_size, n_seq, dim), dtype=float)
                output_y = np.empty((self.batch_size, n_seq, 1), dtype=float)
                #i_input = i * self.batch_size + (k_skip * n_seq)
                i_output_seq = 0
                for i_output_seq in range(self.batch_size) :
                    while i_input + n_seq - 2 in BE['index_end'] :
                        k_skip += 1
                        i_input += n_seq - 1
                    output_X[i_output_batch,i_output_seq] = self.X.iloc[i_input]
                    output_y[i_output_batch,i_output_seq] = self.y.iloc[i_input]                

            yield output_X, output_y
                
    

In [ ]:
# BE 要多新增一個 'length', 作validation時要重算新的BE

In [ ]:
#
# RNN_model
#
def run_RNN_model() :
    
#     data_generator = RNN_data_generator( X, y, BE, n_sen, batch_size=batch_size, n_seq=n_RNN_seq, dim=dim, shuffle = False):
        